In [1]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
import tensorflow as tf # type: ignore
from tensorflow.keras import layers, Model #type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,BatchNormalization,LeakyReLU, Dense, LSTM, Bidirectional, Input, RepeatVector, Concatenate, Dropout #type: ignore
from sklearn.model_selection import StratifiedKFold # type: ignore
from sklearn.utils.class_weight import compute_class_weight # type: ignore
from functools import partial
import math
import matplotlib.pyplot as plt # type: ignore
from tqdm import tqdm # type: ignore

In [2]:
# Constants for image preprocessing
BAND_NAMES = ('B2', 'B3', 'B4', 'B8', 'B11', 'slope')
H, W, NUM_CHANNELS = IMG_DIM = (128, 128, len(BAND_NAMES))
_MAX_INT = np.iinfo(np.uint16).max

def decode_slope(x: np.ndarray) -> np.ndarray:
    # Convert 16-bit discretized slope to float32 radians
    return (x / _MAX_INT * (math.pi / 2.0)).astype(np.float32)

def normalize(x: np.ndarray, mean: int, std: int) -> np.ndarray:
    return (x - mean) / std

rough_S2_normalize = partial(normalize, mean=1250, std=500)

def preprocess_image(x: np.ndarray) -> np.ndarray:
    return np.concatenate([
        rough_S2_normalize(x[..., :-1].astype(np.float32)),
        decode_slope(x[..., -1:]),
    ], axis=-1, dtype=np.float32)


In [3]:
composite_images = np.load('composite_images.npz')
images_path = 'composite_images.npz'
# Load precipitation data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:

train_df['event_id'] = train_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
train_df['event_idx'] = train_df.groupby('event_id', sort=False).ngroup()
test_df['event_id'] = test_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
test_df['event_idx'] = test_df.groupby('event_id', sort=False).ngroup()

train_df['event_t'] = train_df.groupby('event_id').cumcount()
test_df['event_t'] = test_df.groupby('event_id').cumcount()

print(train_df.head())
print(test_df.head())

          event_id  precipitation  label  event_idx  event_t
0  id_spictby0jfsb       0.000000      0          0        0
1  id_spictby0jfsb       0.095438      0          0        1
2  id_spictby0jfsb       1.949560      0          0        2
3  id_spictby0jfsb       3.232160      0          0        3
4  id_spictby0jfsb       0.000000      0          0        4
          event_id  precipitation  event_idx  event_t
0  id_j7b6sokflo4k        0.00000          0        0
1  id_j7b6sokflo4k        3.01864          0        1
2  id_j7b6sokflo4k        0.00000          0        2
3  id_j7b6sokflo4k       16.61520          0        3
4  id_j7b6sokflo4k        2.56706          0        4


In [5]:
# Load composite images
with np.load('composite_images.npz') as data:
    print("Available event IDs in composite_images.npz:", data.keys())
    composite_images = {event_id: data[event_id] for event_id in data.keys()}  # Dictionary mapping event_ids to image arrays

# Preprocess data and images
def preprocess_data_and_images(data_df, composite_images):
    event_ids = data_df['event_id'].unique()
    timeseries = []
    labels = []
    images = []

    for event_id in tqdm(event_ids, desc="Processing data"):
        event_data = data_df[data_df['event_id'] == event_id]
        timeseries.append(event_data['precipitation'].values)  # Shape: (730,)
        if 'label' in event_data.columns:
            labels.append(event_data['label'].values)  # Shape: (730,)
        images.append(preprocess_image(composite_images[event_id]))  # Shape: (128, 128, 6)

    timeseries = np.array(timeseries)
    labels = np.array(labels) if labels else None
    images = np.stack(images, axis=0)

    return timeseries, labels, images


Available event IDs in composite_images.npz: KeysView(NpzFile 'composite_images.npz' with keys: id_rhg5w8vmv3ny, id_rua8ey2jczl0, id_073l04ir88sn, id_wmkfqw7iwjmu, id_heri806er7xw...)


In [6]:
train_timeseries, train_labels, train_images = preprocess_data_and_images(train_df, composite_images)

Processing data: 100%|██████████| 674/674 [00:49<00:00, 13.54it/s]


In [7]:
from tensorflow.keras.losses import binary_crossentropy #type: ignore
def focal_loss(gamma=2., alpha=0.25):
    def loss(y_true, y_pred):
        bce = binary_crossentropy(y_true, y_pred)
        pt = tf.exp(-bce)  # Probabilities of predictions
        focal = alpha * (1 - pt) ** gamma * bce
        return tf.reduce_mean(focal)
    return loss

In [8]:
# Stratified Cross-Validation
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [9]:
# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

In [10]:
# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels.argmax(axis=1)),
    y=train_labels.argmax(axis=1)
)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

In [17]:
for fold, (train_idx, val_idx) in enumerate(kf.split(train_timeseries, train_labels.argmax(axis=1))):
    print(f"Training fold {fold + 1}...")

    X_precip_train, X_precip_val = train_timeseries[train_idx], train_timeseries[val_idx]
    y_train, y_val = train_labels[train_idx], train_labels[val_idx]
    X_img_train, X_img_val = train_images[train_idx], train_images[val_idx]

    # Build the custom neural network
    # Image encoder
    image_input = Input(shape=(128, 128, 6), name='image_input')
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.2)(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    encoded_image = Dense(128, activation='relu')(x)
    # Time-series input
    precip_input = Input(shape=(730,), name='precip_input')
    # Repeat the image encoding vector 730 times and concatenate with precipitation data
    repeated_image_vector = RepeatVector(730)(encoded_image)
    expanded_precip_input = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(precip_input)
    concatenated = Concatenate(axis=-1)([repeated_image_vector, expanded_precip_input])

    # Bidirectional LSTM
    x = Bidirectional(LSTM(64, return_sequences=True))(concatenated)
    x = Dense(64, activation='relu')(x)
    day_probabilities = Dense(1, activation='sigmoid')(x)

    # Define the model
    model = Model(inputs=[image_input, precip_input], outputs=day_probabilities)

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']),
    #model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=0.25), metrics=['accuracy'])

    # Train the model
    model.fit(
        [X_img_train, X_precip_train], y_train,
        validation_data=([X_img_val, X_precip_val], y_val),
        epochs=55,
        callbacks = [early_stopping],
        class_weight=class_weight_dict,
        batch_size=32
    )


c:\Users\Reinhard\anaconda3\envs\Datascience\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Training fold 1...
Epoch 1/55
19/19 ━━━━━━━━━━━━━━━━━━━━ 148s 4s/step - accuracy: 0.8199 - loss: 0.0948 - val_accuracy: 0.9994 - val_loss: 0.0085
Epoch 2/55
19/19 ━━━━━━━━━━━━━━━━━━━━ 73s 4s/step - accuracy: 0.9993 - loss: 0.0059 - val_accuracy: 0.9994 - val_loss: 0.0055
Epoch 3/55
19/19 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.9994 - loss: 0.0048 - val_accuracy: 0.9994 - val_loss: 0.0054
Epoch 4/55
19/19 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.9994 - loss: 0.0049 - val_accuracy: 0.9994 - val_loss: 0.0055
Epoch 5/55
19/19 ━━━━━━━━━━━━━━━━━━━━ 91s 5s/step - accuracy: 0.9993 - loss: 0.0052 - val_accuracy: 0.9994 - val_loss: 0.0055
Epoch 6/55
17/19 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step - accuracy: 0.9993 - loss: 0.0049 

DecodeError: Error parsing message

In [12]:
# Preprocess test data
test_timeseries, _, test_images = preprocess_data_and_images(test_df, composite_images)


Processing data: 100%|██████████| 224/224 [00:05<00:00, 38.03it/s]


In [13]:
# Predict on test set
predictions = model.predict([test_images, test_timeseries])


7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 354ms/step


In [15]:
sample_submission = pd.read_csv('SampleSubmission (2).csv')
sample_submission.head()

,event_id,label
0,id_j7b6sokflo4k_X_0,0
1,id_j7b6sokflo4k_X_1,0
2,id_j7b6sokflo4k_X_2,0
3,id_j7b6sokflo4k_X_3,0
4,id_j7b6sokflo4k_X_4,0


In [16]:
sample_submission['label'] = predictions.flatten()
sample_submission.head()

,event_id,label
0,id_j7b6sokflo4k_X_0,0.023408
1,id_j7b6sokflo4k_X_1,0.018420
2,id_j7b6sokflo4k_X_2,0.017782
3,id_j7b6sokflo4k_X_3,0.017254
4,id_j7b6sokflo4k_X_4,0.017611


In [17]:
sample_submission.to_csv("submission_514.csv", index=False)
